In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
from imblearn.over_sampling import SMOTE
from collections import Counter

In [8]:
df = pd.read_csv('Resources/2020_cleaned.csv')
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealthDays,MentalHealthDays,DifficultyWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivities,GeneralHealth,HoursOfSleep,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [9]:
testing_df = df.copy()
testing_df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealthDays,MentalHealthDays,DifficultyWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivities,GeneralHealth,HoursOfSleep,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310450,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
310451,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
310452,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
310453,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [11]:
testing_cleaned_df = testing_df.drop(columns=['BMI','PhysicalHealthDays','MentalHealthDays','Sex','AgeCategory','Race',
                                              'GeneralHealth','HoursOfSleep'])
testing_cleaned_df

,HeartDisease,Smoking,AlcoholDrinking,Stroke,DifficultyWalking,Diabetic,PhysicalActivities,Asthma,KidneyDisease,SkinCancer
0,No,Yes,No,No,No,Yes,Yes,Yes,No,Yes
1,No,No,No,Yes,No,No,Yes,No,No,No
2,No,Yes,No,No,No,Yes,Yes,Yes,No,No
3,No,No,No,No,No,No,No,No,No,Yes
4,No,No,No,No,Yes,No,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...
310450,Yes,Yes,No,No,Yes,Yes,No,Yes,No,No
310451,No,Yes,No,No,No,No,Yes,Yes,No,No
310452,No,No,No,No,No,No,Yes,No,No,No
310453,No,No,No,No,No,No,No,No,No,No


In [12]:
filtered_df = pd.get_dummies(testing_cleaned_df,dtype=float)
filtered_df.head(10)

,HeartDisease_No,HeartDisease_Yes,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,Stroke_Yes,DifficultyWalking_No,DifficultyWalking_Yes,Diabetic_No,Diabetic_Yes,PhysicalActivities_No,PhysicalActivities_Yes,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
5,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
7,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
9,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [13]:
filtered_df = filtered_df.drop(columns=['HeartDisease_No','Smoking_No','AlcoholDrinking_No','Diabetic_No','Stroke_No',
                                       'DifficultyWalking_No','PhysicalActivities_No','Asthma_No',
                                       'KidneyDisease_No','SkinCancer_No'])
filtered_df

,HeartDisease_Yes,Smoking_Yes,AlcoholDrinking_Yes,Stroke_Yes,DifficultyWalking_Yes,Diabetic_Yes,PhysicalActivities_Yes,Asthma_Yes,KidneyDisease_Yes,SkinCancer_Yes
0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
310450,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
310451,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
310452,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
310453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X = filtered_df.copy()
X.drop(columns={'HeartDisease_Yes'},axis=1, inplace=True)
X.head()

,Smoking_Yes,AlcoholDrinking_Yes,Stroke_Yes,DifficultyWalking_Yes,Diabetic_Yes,PhysicalActivities_Yes,Asthma_Yes,KidneyDisease_Yes,SkinCancer_Yes
0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [15]:
y = filtered_df['HeartDisease_Yes'].ravel()
y[:5]

array([0., 0., 0., 0., 0.])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
smt = SMOTE()

In [17]:
counter = Counter(y_train)
print(counter)

Counter({0.0: 212986, 1.0: 19855})


In [ ]:
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

In [ ]:
counter = Counter(y_train_sm)
print(counter)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaler = scaler.fit(X_train_sm)

In [ ]:
X_train_scaled = X_scaler.transform(X_train_sm)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=1567)

In [ ]:
rf_model = rf_model.fit(X_train_scaled, y_train_sm)

In [ ]:
predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)